<a href="https://colab.research.google.com/github/TokaevAD/Cuda/blob/main/Cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Mar 13 11:06:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [3]:
!pip install numba


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from numba import jit
import numpy as np
from timeit import default_timer as timer
import random
from numba import cuda

Задание 2

In [5]:
@jit
def funcGPU(x):
  b = np.zeros(x.shape)
  for i in range(1, x.shape[0]-1):
    b[i-1] = (x[i-1]+x[i]+x[i+1])/3
  return b

In [6]:
def func(x):
  b = np.zeros(x.shape)
  for i in range(1, x.shape[0]-1):
    b[i-1] = (x[i-1]+x[i]+x[i+1])/3
  return b 

In [7]:
if __name__=="__main__":
    n = 16000
    a = np.array([i for i in range(n)])
    start = timer()
    func(a)
    print("не используя GPU:", timer()-start)
    start = timer()
    funcGPU(a)
    cuda.profile_stop()
    print("используя GPU:", timer()-start)

не используя GPU: 0.019426937999980964
используя GPU: 1.567477591999932


Задание 4

In [8]:
def matrix_multiplication(matrix1, matrix2):

    # Проверяем, что матрицы можно перемножить
    if len(matrix1[0]) != len(matrix2):
        raise ValueError("Количество столбцов первой матрицы должно совпадать с количеством строк второй матрицы")

    # Создаем результирующую матрицу, заполненную нулями
    result_matrix = [[0 for j in range(len(matrix2[0]))] for i in range(len(matrix1))]

    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            scalar_product = 0
            for k in range(len(matrix2)):
                scalar_product += matrix1[i][k] * matrix2[k][j]

            result_matrix[i][j] = scalar_product

    return result_matrix


In [9]:
@jit
def matrix_multiplicationGPU(matrix1, matrix2):

    if len(matrix1[0]) != len(matrix2):
        raise ValueError("Количество столбцов первой матрицы должно совпадать с количеством строк второй матрицы")

    result_matrix = [[0 for j in range(len(matrix2[0]))] for i in range(len(matrix1))]

    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            scalar_product = 0
            for k in range(len(matrix2)):
                scalar_product += matrix1[i][k] * matrix2[k][j]

            result_matrix[i][j] = scalar_product

    return result_matrix

In [12]:
if __name__=="__main__":
    m, n, k = 1000, 1000, 1000
    matrix1 = np.random.randint(100, size = (m, n))
    matrix2 = np.random.randint(100, size = (n, k))
    start = timer()
    matrix_multiplication(matrix1, matrix2)
    print("не используя GPU:", timer()-start)
    start = timer()
    matrix_multiplicationGPU(matrix1, matrix2)
    cuda.profile_stop()
    print("используя GPU:", timer()-start)

не используя GPU: 673.7963576510001
используя GPU: 1.3906651000002057


Задание 1

In [13]:
@jit
def HelloWorld():
  print("Hello world")

In [14]:
if __name__=="__main__":
  HelloWorld()

Hello world


Лабораторная работа

In [3]:
import math

def f(x):
  return np.exp(x)

In [4]:
def rectangle_rule_integration(a, b, n):
    """
    Вычисляет определенный интеграл f(x) на интервале [a, b]
    с использованием метода прямоугольников с n прямоугольниками.

    f - функция, которую необходимо проинтегрировать
    a, b - конечные точки интервала интегрирования
    n - количество прямоугольников
    """
    h = (b - a) / n  # ширина прямоугольника
    integral_sum = 0
    for i in range(n):
        x_i = a + i * h  # левая граница i-го прямоугольника
        integral_sum += f(x_i) * h
    return integral_sum


In [5]:
@jit
def rectangle_rule_integrationGPU(a, b, n):
    
    h = (b - a) / n 
    integral_sum = 0
    for i in range(n):
        x_i = a + i * h  
        integral_sum += f(x_i) * h
    return integral_sum


In [ ]:
if __name__=="__main__":
    n = 10000000
    a = 0
    b = 1
    print("n =",n)
    start = timer()
    rectangle_rule_integration(a,b,n)
    print("не используя GPU:", timer()-start)
    start = timer()
    rectangle_rule_integrationGPU(a,b,n)
    cuda.profile_stop()
    print("используя GPU:", timer()-start)


n = 10000\
не используя GPU: 0.021303477999026654\
используя GPU: 0.023017559999061632\
n = 100000\
не используя GPU: 0.12404612700083817\
используя GPU: 0.1192904230010754\
n = 10000000\
не используя GPU: 12.443151404999298\
используя GPU: 12.2176247339994